In [1]:
from openai import OpenAI

In [2]:
# Connection parameters (only has to be executed once during a session)

# Note: `localhost` is only valid as long as you are logged in on madagaskarweihe. 
# Otherwise, you will need to perform port forwarding on your machine using `ssh -fNL 8006:127.0.0.1:8006 ashousaa@madagaskarweihe`.
# This might require an ssh key to be uploaded to madagaskarweihe first.
VLLM_API_ENDPOINT = 'http://localhost:8006/v1' 
VLLM_KEY = 's7Vtzeyq3kfhVkPhlWdL95pPRBq36KDP1d5bBj54BqQ'
MODEL = "default"

In [3]:
# Connection to LLM server (only has to be executed once during a session)
client = OpenAI(api_key=VLLM_KEY,
                base_url=VLLM_API_ENDPOINT)

In [4]:
# Get the top-k matching chunks for a given query

from sentence_transformers import SentenceTransformer
import torch
import pandas as pd

def get_top_matching_chunks(query, k):
    # Load the model only once (move it outside this function if calling often)
    model = SentenceTransformer("all-MiniLM-L6-v2")

    # Load CSV with both chunk_id and chunk_text
    df = pd.read_csv("chunks.csv")  # assumes columns: chunk_id, chunk_text

    # Get lists
    chunk_ids = df["chunk_id"].tolist()
    chunk_texts = df["chunk_text"].tolist()

    # Encode chunks and query
    chunk_embeddings = model.encode(chunk_texts, convert_to_tensor=True)
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarity
    cos = torch.nn.CosineSimilarity(dim=1)
    similarities = cos(chunk_embeddings, query_embedding)

    # Get top-k indices
    top_indices = torch.topk(similarities, k=k).indices.tolist()


    # Map indices back to chunk_ids
    top_chunk_ids = [chunk_ids[i] for i in top_indices]

    return top_chunk_ids


/mount/arbeitsdaten/studenten4/ashousaa/.env/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# This method reads a CSV file containing questions and their corresponding chunk IDs.
# It returns a list of dictionaries where each dictionary contains the chunk ID as the key and the question as the value.

import csv

def get_queries_from_csv(csv_file):
    queries_chunk_map = []

    # Open and read the CSV file
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)

        # Iterate over each row in the CSV file
        for row in reader:
            chunk_id = row['Chunk']  # Assuming 'chunk_id' column is present
            query= row['Question']  # Assuming 'question' column is present

            # Create a dictionary with chunk_id as key and question as value
            queries_chunk_map.append({chunk_id: query})

    return queries_chunk_map

In [6]:
import pandas as pd
def get_CSVcolumn(csv_file,column_name):


    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Extract the questions into a list
    column_list = df[column_name].tolist()
    
    # Return the list
    return column_list

In [9]:
import unicodedata
import re

def normalize_string(s):
    # Step 1: Convert to lowercase
    s = s.lower()
    
    # Step 2: Strip leading/trailing whitespace
    s = s.strip()
    
    # Step 3: Normalize Unicode characters (e.g., accented characters)
    s = unicodedata.normalize('NFKD', s)  # Normalize to decomposed form
    
    # Step 4: Remove non-printable characters (e.g., invisible characters)
    s = re.sub(r'[^\x20-\x7E]', '', s)  # Keep only printable characters
    
    return s

In [ ]:
import csv

k = 10
queries_chunk_map = get_queries_from_csv("final_test_set.csv")[:20]  # Limit to 20 queries
csv_file = '/mount/arbeitsdaten/studenten4/ashousaa/chunks.csv'
column_name = 'chunk_text'
chunk_texts = get_CSVcolumn(csv_file, column_name)  

# List of prompts to try
prompts = [
    "Given the following facts: {top_k_string}, base your answer on the information provided in the documents. Do not include any additional information or context. Answer the question as precisely as possible.",
    "Given the following facts: {top_k_string}, base your answer only on the provided documents. Answer precisely.",
    "I want you to answer the question based on the retrieved context below.",
    "Based on the following retrieved documents {top_k_string}, please answer the question. Think step by step about how the documents relate to the question before providing your final answer.",
    "Answer the question using only the provided context {top_k_string}. If the answer is not in the context, say so.Be concise by extracting only the relevant information.",
    "Answer the question using only the context below {top_k_string}; if additional information is needed, specify what is missing in your response.",
    "Use the following pieces of retrieved context to answer the question {top_k_string} . If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.",
    "Given the context information {top_k_string} and not prior knowledge, answer the query.",
    "Answer the question using only the context provided.",
    "Extract the most relevant passage from the retrieved documents {retrieved documents} that answers the query {query}. Return only the exact text from {retrieved documents} without modification.",
    "Answer the users QUESTION using the DOCUMENT text above. Keep your answer grounded in the facts of the DOCUMENT. If the DOCUMENT doesn’t contain the facts to answer the QUESTION return {NONE}"
]

all_rows = []  # Collect all rows to write later

# Loop over prompts
for prompt_index, prompt_template in enumerate(prompts, start=1):
    prompt_name = f"Prompt {prompt_index}"
    prompt_rows = []  # Collect rows for this prompt
    correct_answers_count = 0  # Counter for correct answers in this prompt

    for i, query_map in enumerate(queries_chunk_map, start=1):
        chunk_id, query = list(query_map.items())[0]  # Extract chunk ID and question

        # Get top k chunks
        top_k_chunk_ids = get_top_matching_chunks(query, k)

        # Prepare top-k documents
        top_k_documents = [chunk_texts[int(chunk_id.split("_")[1]) - 1] for chunk_id in top_k_chunk_ids]
        top_k_string = "\n".join(top_k_documents)

        # Fill in prompt template
        system_prompt = prompt_template.format(top_k_string=top_k_string)
        print(f"Prompt: {system_prompt}")

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"What is the best answer to the question: {query}?"}
        ]

        # Get answer from LLM
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            seed=42,
            temperature=0.7,
        )

        raw_response = response.choices[0].message.content.strip()

        # Check if correct chunk is among top-k
        normalized_chunk_id = normalize_string(chunk_id)
        normalized_top_k_chunk_ids= [normalize_string(a) for a in top_k_chunk_ids]
        correct_chunk_found = normalized_chunk_id in normalized_top_k_chunk_ids

        if correct_chunk_found:
            correct_answers_count += 1

        # Save row, without total_correct_answers_for_prompt for now
        prompt_rows.append([
            prompt_name,
            i,
            query,
            chunk_id,
            "; ".join(top_k_chunk_ids),
            correct_chunk_found,
            raw_response,
            None  # Placeholder, will fill later
        ])

    # After all questions for this prompt, set the total correct
    for row in prompt_rows:
        row[-1] = correct_answers_count  # Set in last column

    all_rows.extend(prompt_rows)  # Add these rows to the final table

# Write everything to CSV
with open("RAG_Test_Results.csv", mode="w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow(["Prompt", "Question Number", "Question Text", "Original Chunk ID", "Top-k Chunk IDs", "Correct Chunk Found", "Generated Answer", "Total Correct Answers For This Prompt"])
    writer.writerows(all_rows)

print("Finished! All results saved to RAG_Test_Results.csv ✅")


Prompt: Given the following facts: State Travel Expense Act 
(LRKG) §  Official Trips and Errands () Official trips in the sense of this Act are journeys 
undertaken to conduct official business outside the usual place of work, which have been 
ordered or approved by the responsible superior, unless an order or approval is not applicable 
due to the nature of the official’s office or the nature of the official business. The order or 
approval must be given in writing or electronically. Official trips also include journeys from a 
location serving as a temporary residence to the place of work, provided that the conditions of 
sentences  and  are otherwise met. Official trips should only be carried out if a less costly 
method of conducting the official business is not possible or reasonable.
State Travel Expense 
Act (LRKG) §  Official Trips and Errands () Official errands are journeys undertaken to conduct 
official business at the place of work or residence outside the official premis